# BBLF AI Selector: Part 2: During Tournament Optimal Squad - Optimser (Round 8)

# 0. Prerequistes

In [7]:
# 0. Prerequistes

import pandas as pd
import numpy as np
import os
import random
from mip import Model, xsum, maximize, BINARY 

os.getcwd()
directory = 'C:/Users/dilan/OneDrive/Documents/Data Science Projects/Big Bash Fantasy AI'
add_data_directory = 'C:/Users/dilan/OneDrive/Documents/Data Science Projects/Big Bash Fantasy AI/data/add_data_created/round_8'
over_data_directory = 'C:/Users/dilan/OneDrive/Documents/Data Science Projects/Big Bash Fantasy AI/data/add_data_created/overall'
py_data_directory = 'C:/Users/dilan/OneDrive/Documents/Data Science Projects/Big Bash Fantasy AI/data/python_datasets'

# 1. Data Extraction 

In [8]:
# 1. Data Extraction 
    # Current Round & Optimal Number of Rounds
current_round = 8
opt_round = 2

    # Pull in player price data csv file 

price_df = pd.read_csv(os.path.join(add_data_directory,'player_price_rnd_8.csv'), low_memory=False)
#price_df = pd.read_csv(os.path.join(add_data_directory,'player_price_rnd_', str(current_round), '.csv'), low_memory=False)

# print(price_df) 

    # Player Role Flags

price_df["Wk_f"] = np.where((price_df["Role"] == "WK"), 1, 0)
price_df["Bat_f"] = np.where((price_df["Role"] == "WK") |(price_df["Role"] == "BAT") | (price_df["Role"] == "ALLR") , 1, 0)
price_df["Bowl_f"] = np.where((price_df["Role"] == "BOWL") | (price_df["Role"] == "ALLR") , 1, 0)
price_df = price_df[["Full_Name", "Price", "Team","Wk_f", "Bat_f", "Bowl_f", "Role", "Available", "In_Team"]].rename(columns = {"Full_Name": "Name"}) 
    
    # Pull in team fixture csv file

team_fix_df = pd.read_csv(os.path.join(over_data_directory,'team_loc_fixture.csv'), low_memory=False)
team_fix_df = team_fix_df[team_fix_df.Round >= current_round].dropna()
team_fix_df = team_fix_df[team_fix_df.Round <= current_round + opt_round - 1].dropna()
# print(team_fix_df)

    # Pull player expected points csv file
exp_pts_df = pd.read_csv(os.path.join(py_data_directory,'bbl14_fullteam_model_score_during_rnd_8.csv'), low_memory=False).drop(["Unnamed: 0", "player"], axis = 1)
# print(exp_pts_df)

    # Join team fixture to player price to create a row for each game
game_df = pd.merge(price_df , team_fix_df, left_on = ["Team"], right_on = ["Team"], how = "left")
# print(game_df)

    # Join on expected points for each game
# print(game_df.columns)
# print(exp_pts_df.columns)
player_df_raw = pd.merge(game_df, exp_pts_df, left_on = ["Name", "Team", "Opposition", "Venue", "Home_f","Round"], right_on = ["Name", "Team", "Opposition", "Venue", "Home_f","Round"], how = "left")
#player_df_raw["exp_pts"] = np.where((player_df_raw["Role"] == "WK"), player_df_raw["exp_pts"] + 10.51, player_df_raw["exp_pts"] + 4.53)
player_df_raw["weight"] = 1
# print(player_df_raw)

    # Aggregate by player name (calculate total expected points for each player for x number of rounds)
player_df = player_df_raw.groupby(['Name', 'Price', "Team", "Wk_f", "Bat_f", "Bowl_f", "Role", "weight","Available", "In_Team"], as_index=False).agg(
exp_pts=('exp_pts',"sum"))
# print(player_df)

player_df_next_rnd = player_df_raw[player_df_raw.Round == current_round].groupby(['Name', 'Price', "Team", "Wk_f", "Bat_f", "Bowl_f", "Role", "weight","Available", "In_Team"], as_index=False).agg(
next_rnd_exp_pts=('exp_pts',"sum"))
player_df_next_rnd = player_df_next_rnd[["Name", "Team", "next_rnd_exp_pts"]]


# 2. Optimisation Process (Round 8) - Normal 3 Trades

Optimisation Objective: Maximise the number of expected fantasy points

Constraints: 
1. Number of players selected must be 12
2. Atleast 1 wicketkeeper
3. Atleast 6 batters
4. Atleast 5 bowlers
5. Total budget of team is less than $1,687,000
6. Atleast 9 players in current team (3 trades)

In [9]:
# Optimisation Setup

points = player_df["exp_pts"]

price = player_df["Price"]

#model_df.loc[:,"opp_Adelaide Strikers":"venue_WACA"] = model_df.loc[:,"opp_Adelaide Strikers":"venue_WACA"].astype(object)

weight = player_df["weight"]
available = player_df["Available"]
in_team = player_df["In_Team"]
wk_weight = player_df["Wk_f"]
bat_weight = player_df["Bat_f"]
bowl_weight = player_df["Bowl_f"]

play_cnt, total_player = 12, range(len(price))
team_play_cnt, total_team_player = 9, range(len(price))
wk_cnt, total_wk = 1, range(len(price))
bat_cnt, total_bat = 6, range(len(price))
bowl_cnt, total_bowl = 5, range(len(price))
budget, total_budget = 1687000, range(len(price))

m = Model("knapsack")

x = [m.add_var(var_type=BINARY) for i in total_player]
print(x)

m.objective = maximize(xsum(points[i]*x[i] for i in total_player))

m += xsum(weight[i] * x[i] for i in total_player) == play_cnt
m += xsum(wk_weight[i] * x[i] for i in total_wk) >= wk_cnt
m += xsum(bat_weight[i] * x[i] for i in total_bat) >= bat_cnt
m += xsum(bowl_weight[i] * x[i] for i in total_bowl) >= bowl_cnt
m += xsum(price[i] * x[i] for i in total_budget) <= budget
m += xsum(available[i] * x[i] for i in total_player) == play_cnt
m += xsum(in_team[i] * x[i] for i in total_team_player) >= team_play_cnt

m.optimize()

selected = [i for i in total_player if x[i].x >= 0.99]
print("selected items: {}".format(selected))

sel_player_df = player_df.iloc[selected]
sel_player_df = pd.merge(sel_player_df, player_df_next_rnd, left_on = ["Name","Team"], right_on = ["Name","Team"], how = "left")
print("Total Expect Points:", sum(sel_player_df["exp_pts"]))
print("Total Next Round Points:", sum(sel_player_df["next_rnd_exp_pts"]))
print("Total Team Cost:", sum(sel_player_df["Price"]))
print("Number of Wk:", sum(sel_player_df["Wk_f"]))
print("Number of Bat:", sum(sel_player_df["Bat_f"]))
print("Number of Bowl:", sum(sel_player_df["Bowl_f"]))
print("Available Players:", sum(sel_player_df["Available"]))
print("Current Players Remaining:", sum(sel_player_df["In_Team"]))

print(sel_player_df.sort_values(by = "Price", ascending = False))
print(sel_player_df[sel_player_df['In_Team'] == 1].sort_values(by = "Price", ascending = False))
print(sel_player_df[sel_player_df['In_Team'] == 0].sort_values(by = "Price", ascending = False))


sel_player_df.to_csv('C:/Users/dilan/OneDrive/Documents/Data Science Projects/Big Bash Fantasy AI/data/python_datasets/optimal_round_8.csv')


[<mip.entities.Var object at 0x000001C92BC8E380>, <mip.entities.Var object at 0x000001C92BC8C130>, <mip.entities.Var object at 0x000001C92BC8D930>, <mip.entities.Var object at 0x000001C92BC8C4F0>, <mip.entities.Var object at 0x000001C92BC8D3C0>, <mip.entities.Var object at 0x000001C92BC8D240>, <mip.entities.Var object at 0x000001C92BC8CE50>, <mip.entities.Var object at 0x000001C92BC8D2D0>, <mip.entities.Var object at 0x000001C92BC8CF70>, <mip.entities.Var object at 0x000001C92BC8C910>, <mip.entities.Var object at 0x000001C92BC8D360>, <mip.entities.Var object at 0x000001C92BC8CFD0>, <mip.entities.Var object at 0x000001C92BC8F0A0>, <mip.entities.Var object at 0x000001C92BC8CB50>, <mip.entities.Var object at 0x000001C92BC8C1F0>, <mip.entities.Var object at 0x000001C92BC8F130>, <mip.entities.Var object at 0x000001C92BC8E830>, <mip.entities.Var object at 0x000001C92BC8F2E0>, <mip.entities.Var object at 0x000001C92BC8F190>, <mip.entities.Var object at 0x000001C92BC8F1F0>, <mip.entities.Var o